In [20]:
import tensorflow as tf
import numpy as np
import cv2

from skimage.transform import resize
from tensorflow.keras.layers import TimeDistributed, LSTM, Dense, Dropout, Flatten, GlobalAveragePooling1D
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.models import Sequential

In [22]:
def model(tf, weights='weights.hdfs'):
    layers = tf.keras.layers
    models = tf.keras.models
    losses = tf.keras.losses
    optimizers = tf.keras.optimizers
    metrics = tf.keras.metrics
    num_classes = 2
    cnn = models.Sequential()
    #cnn.add(base_model)

    input_shapes=(160,160,3)
    np.random.seed(1234)
    vg19 = tf.keras.applications.vgg19.VGG19
    base_model = vg19(include_top=False,weights='imagenet',input_shape=(160, 160,3))
    # Freeze the layers except the last 4 layers
    #for layer in base_model.layers:
    #    layer.trainable = False

    cnn = Sequential()
    cnn.add(base_model)
    cnn.add(Flatten())
    model = Sequential()

    model.add(TimeDistributed(cnn,  input_shape=(30, 160, 160, 3)))
    model.add(LSTM(30 , return_sequences= True))

    model.add(TimeDistributed(Dense(90)))
    model.add(Dropout(0.1))

    model.add(GlobalAveragePooling1D())

    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.3))

    model.add(Dense(num_classes, activation="sigmoid"))

    adam = Adam(lr=0.0005, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    model.load_weights(wight)
    rms = RMSprop()

    model.compile(loss='binary_crossentropy', optimizer=adam, metrics=["accuracy"])

    return model


In [23]:
np.random.seed(1234)
uploaded_model = model(tf)


In [24]:
def video_mamonreader(cv2, filename):
    frames = np.zeros((30, 160, 160, 3), dtype=np.float)
    i=0
    print(frames.shape)
    vc = cv2.VideoCapture(filename)
    if vc.isOpened():
        rval , frame = vc.read()
    else:
        rval = False
    frm = resize(frame,(160,160,3))
    frm = np.expand_dims(frm,axis=0)
    if(np.max(frm)>1):
        frm = frm/255.0
    frames[i][:] = frm
    i +=1
    print("reading video")
    while i < 30:
        rval, frame = vc.read()
        frm = resize(frame,(160,160,3))
        frm = np.expand_dims(frm,axis=0)
        if(np.max(frm)>1):
            frm = frm/255.0
        frames[i][:] = frm
        i +=1
    return frames

In [32]:
def predictions(model, video, confidence_level=0.9):
    prediction = model.predict(video)
    if prediction[0][1] >= confidence_level:
        return True , prediction[0][1]
    else:
        return False , prediction[0][1]

In [41]:
def main_fight(video):
    video = video_mamonreader(cv2, video)
    input_data = np.zeros((1, 30, 160, 160, 3), dtype=np.float)
    input_data[0][:][:] = video
    boolean_output, precent = predictions(uploaded_model, input_data, confidence_level=0.65)
    result = f"fight:{boolean_output}, probability: {precent}"
    return result

In [42]:
res = main_fight('1.mp4')

(30, 160, 160, 3)
reading video


In [43]:
res

'fight:False, probability: 0.009498625993728638'

In [36]:
res = main_fight('input.mp4')

(30, 160, 160, 3)
reading video


In [37]:
res

'fight:False, probability: 0.4736264944076538'

In [67]:
def predicts(video, confidence_level=0.9):
    prediction = model.predict(video)
    if prediction[0][1] >= confidence_level:
        return True, prediction[0][1]
    else:
        return False, prediction[0][1]

In [68]:
num_classes = 2

cnn = Sequential()
input_shapes=(160,160,3)
np.random.seed(1234)
base_model = tf.keras.applications.vgg19.VGG19(include_top=False, weights='imagenet', input_shape=(160, 160,3))

cnn = Sequential()
cnn.add(base_model)
cnn.add(Flatten())
model = Sequential()

model.add(TimeDistributed(cnn,  input_shape=(30, 160, 160, 3)))
model.add(LSTM(30 , return_sequences= True))

model.add(TimeDistributed(Dense(90)))
model.add(Dropout(0.1))

model.add(GlobalAveragePooling1D())

model.add(Dense(512, activation='relu'))
model.add(Dropout(0.3))

model.add(Dense(num_classes, activation="sigmoid"))

adam = Adam(lr=0.0005, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.load_weights("weights.hdfs")
rms = RMSprop()

model.compile(loss='binary_crossentropy', optimizer=adam, metrics=["accuracy"])


In [69]:
model.summary()

Model: "sequential_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_14 (TimeDis (None, 30, 12800)         20024384  
_________________________________________________________________
lstm_7 (LSTM)                (None, 30, 30)            1539720   
_________________________________________________________________
time_distributed_15 (TimeDis (None, 30, 90)            2790      
_________________________________________________________________
dropout_14 (Dropout)         (None, 30, 90)            0         
_________________________________________________________________
global_average_pooling1d_7 ( (None, 90)                0         
_________________________________________________________________
dense_22 (Dense)             (None, 512)               46592     
_________________________________________________________________
dropout_15 (Dropout)         (None, 512)             

In [70]:
cap = cv2.VideoCapture(0)
while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    frames = resize(frame, output_shape=(160,160,3))
    input_data = np.zeros((1, 30, 160, 160, 3), dtype=np.float)
    input_data[0][:][:] = frames
    f , percent = predicts(input_data, confidence_level=0.99)
    print(f"Fight: {f}, confidence {percent}")
    # Our operations on the frame come here
    # Display the resulting frame
    cv2.imshow('frame',frames)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

Fight: False, confidence 0.6637696623802185
Fight: False, confidence 0.6592472791671753
Fight: False, confidence 0.6601521372795105
Fight: False, confidence 0.6665381193161011
Fight: False, confidence 0.6614807844161987
Fight: False, confidence 0.9695391654968262
Fight: False, confidence 0.9134311676025391
Fight: False, confidence 0.9644286632537842
Fight: False, confidence 0.9779243469238281
Fight: False, confidence 0.5177187919616699
Fight: False, confidence 0.7999491095542908
Fight: False, confidence 0.7185858488082886
Fight: False, confidence 0.29327744245529175
Fight: False, confidence 0.6552181839942932
Fight: False, confidence 0.9676059484481812
